In [1]:
import numpy as np
import scipy as sp
from scipy.io import loadmat
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from skimage import data
from skimage import transform
from skimage.color import rgb2gray
from skimage.feature import match_descriptors, plot_matches, SIFT

# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'

In [2]:
path = r'C:\Users\erikn\skola\EEN020-Computer-Vision\assignment-2/'

img1 = r'A2data\data/cube1.jpg'
cube1 = cv.load_image(path+img1)

img2 = r'A2data\data/cube2.jpg'
cube2 = cv.load_image(path+img2)

In [3]:
def get_sift_plot_points(img1_pts, img2_pts, img1):
    x = [img1_pts[0,:], np.size(img1,1)+img2_pts[0,:]]
    y = [img1_pts[1,:], img2_pts[1,:]]
    return x, y

In [4]:
def compute_sift_points(img1, img2, marg):
    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary

    flann = cv2.FlannBasedMatcher(index_params, search_params)
    matches = flann.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    draw_params = dict(matchColor=(255,0,255), singlePointColor=(0,255,0), matchesMask=None, flags=cv2.DrawMatchesFlags_DEFAULT)
    img_match = cv2.drawMatchesKnn(img1, kp1, img2, kp2, good_matches, None, **draw_params)

    img1_pts = np.stack([kp1[match[0].queryIdx].pt for match in good_matches],1)
    img2_pts = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)

    print('Number of good matches:', np.size(img1_pts,1))

    return img1_pts, img2_pts, img_match

In [5]:
cube1_pts, cube2_pts, cube_match = compute_sift_points(cube1, cube2, 2)

# np.save(path+r'A2data\data/cube1_pts.npy', cube1_pts)
# np.save(path+r'A2data\data/cube2_pts.npy', cube2_pts)

x, y = get_sift_plot_points(cube1_pts, cube2_pts, cube1)

plt.figure(figsize=(20,9))
plt.plot(x, y, 'o-', ms=5, lw=1, color='magenta')
plt.xlabel('$x$')
plt.ylabel('$y$')
plt.axis('equal')
plt.imshow(cv2.cvtColor(cube_match, cv2.COLOR_BGR2RGB))
plt.show()

Number of good matches: 6412
